In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
air = pd.read_csv('../../data/processed/aggregated_aqi_daily_max.csv')
# bike = pd.read_csv('../../data/processed/aggregated_bikes.csv')
bike = pd.read_csv('../../data/processed/aggregated_bikes_bsfa.csv')
sta = pd.read_csv('../../data/processed/sta.csv')
weather = pd.read_csv('../../data/processed/weather.csv')

In [ ]:
#bike.shape
bike.shape

In [ ]:
bike

In [ ]:
# NEW CODE 
bike['user_type_clean'] = np.where(bike['bike_share_for_all_trip'] == 'Yes', 'Subscriber - BSFA', 
                                        np.where((bike['user_type'] == 'Subscriber') & (bike['bike_share_for_all_trip'] == 'No'), 'Subscriber - Standard',
                                            'Customer'))



In [ ]:
bike.groupby(["user_type_clean", "user_type", 'bike_share_for_all_trip']).size()

In [ ]:
# NEW CODE 
bike = bike.drop(['user_type', 'bike_share_for_all_trip'], axis=1) \
    .rename({'user_type_clean': 'user_type'})

In [ ]:
air.shape

In [ ]:
sta.shape

In [ ]:
weather.shape

In [ ]:
sta.groupby('date_clean').agg(n = ('date_clean', 'count')).sort_values('n')

In [ ]:
air

In [ ]:
weather

In [ ]:
df = bike.merge(air, how='left', left_on='start_date', right_on='date')
air_bike_size = df.shape[0]


In [ ]:
df = df.merge(sta, how='left', left_on='date', right_on='date_clean')
air_bike_sta_size = df.shape[0]
print(f'Size of new ({air_bike_sta_size}) same as old ({air_bike_size})? -> {air_bike_sta_size == air_bike_size}')

In [ ]:
df = df.merge(weather, how='left', left_on='date', right_on='time')
air_bike_sta_weather_size = df.shape[0]
print(f'Size of new ({air_bike_sta_weather_size}) same as old ({air_bike_size})? -> {air_bike_sta_weather_size == air_bike_size}')

In [ ]:
df.groupby('date').agg(n = ('date', 'count')).sort_values('n')

In [ ]:
df['sta'] = df['sta'].fillna(0)

In [ ]:
df.groupby('date').agg(n=('date', 'count')).reset_index().groupby('n').count()

In [ ]:
df_complete = df[~df['start_date'].isna()]
df_complete.shape

In [ ]:
df_complete.apply(lambda x: sum(x.isna()), axis=0)

NOTE FOR PAPER: wpgt and tsun are missing. So they are dropped. 

In [ ]:
df_complete['PM2.5'] = df_complete['PM2.5'].fillna(df_complete['PM2.5'].mean())
df_complete['pres'] = df_complete['pres'].fillna(df_complete['pres'].mean())
df_complete['snow'] = df_complete['snow'].fillna(df_complete['snow'].mean())
df_complete['day_of_week'] = pd.to_datetime(df_complete['start_date']).dt.day_of_week

In [ ]:
df_complete = df_complete.drop(['date_clean', 'date', 'wpgt', 'tsun', 'time', 'wdir'], axis=1)

In [ ]:
df_complete.apply(lambda x: sum(x.isna()), axis=0)

In [ ]:
df_complete.columns

In [ ]:
# NEW CODE - RESIDUALS
# TODO - impute missing values
rolling_cols = ['num_trips', 'avg_duration', 'max_aqi', 'max_aqi_category', 'CO', 'NO2', 'OZONE', 'PM2.5', 'tavg', 'tmin', 'tmax', 'wspd', 'pres']
df_complete = df_complete.sort_values(['user_type_clean', 'start_date']).reset_index()
df_complete_rolling = df_complete.groupby(['user_type_clean', 'day_of_week']) \
    .rolling(window=9, center=True).mean() \
    .reset_index()

df_complete_rolling.set_index('level_2', inplace=True)
df_complete_rolling.sort_index(inplace=True)

residuals = (df_complete[rolling_cols] - df_complete_rolling[rolling_cols]) / df_complete_rolling[rolling_cols]
residuals = residuals.add_suffix('_res')

In [ ]:
# join residuals onto df_complete
# select columns of interest
df_complete_residuals = df_complete.merge(residuals, left_index=True, right_index=True) \
    .dropna()

In [ ]:
df_complete_residuals.apply(lambda x: sum(x.isna()), axis=0)

In [ ]:
df_complete_residuals.head(20)

In [ ]:
df_complete_rolling.head(20)

In [ ]:
df_complete.head(20)

In [ ]:
residuals.apply(lambda x: sum(x.isna()), axis=0)

In [ ]:
residuals_complete = residuals.dropna()
residuals_complete.head(20)

In [ ]:
df_complete.head(20)

In [ ]:
# NEW CODE - TIME LAG
# 
df_complete_lags = df_complete[rolling_cols].assign(**{
    f'{col} (t-{1})': df[col].shift(1)
    for col in df
})
    

In [ ]:
df_complete_lags[697:]

In [ ]:
df_complete.head(20)

In [ ]:
residuals.head(20)

In [ ]:
df_complete_rolling[df_complete_rolling['user_type_clean'] == 'Subscriber - BSFA'].head(20)

In [ ]:
len(df_complete.columns)

In [ ]:
df[~df['start_date'].isna()].sort_values('date')